In [1]:
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path
import scipy.signal as signal
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

cwd = os.getcwd()
sys.path.insert(0, os.path.abspath(os.path.dirname(cwd)))

import phonlab as phon
print(phon.__name__,phon.__version__)
print(phon.get_timestamp_now())

from audiolabel import read_label   # Ronald Sprouse's audiolabel library - to read TextGrid files


phonlab 0.0.1
('2024-11-14T151906', '-0800')


In [2]:
mandarin = Path(
    '/Volumes/KJohnson/Corpora/asc/corpus'
)

# get a dataframe listing all of the TextGrid files in the database
tgdf = phon.dir2df(mandarin, fnpat='.TextGrid$', addcols=['barename'])
print(f'Found {len(tgdf)} TextGrid files.')

tgdf

Found 0 TextGrid files.


,relpath,fname,barename


In [3]:
speaker=""
rng = np.random.default_rng()

bigdf = pd.DataFrame()

for row in tgdf.itertuples():   # loop through all of the label files
    if rng.random()<0.75:
        continue
    
    if speaker != row.relpath:
        speaker = row.relpath
        print(f"{speaker}")   


    wav = f'{row.barename}.wav'
    wavname = str(mandarin / row.relpath / wav)  # construct the name of the wav file

    f0df = phon.get_f0(wavname, chan = 0, pre = 0.96)
    eggdf = phon.egg2oq(wavname, egg_channel=1,hop_dur = 0.01)
    f0df['vuv'] = 'uv'
    f0df['vuv'] = f0df['vuv'].where(np.isnan(eggdf.OQ), 'v')
    f0df['file'] = wav

    if bigdf.empty:
        bigdf = f0df
    else:
        bigdf = pd.concat([bigdf,f0df],ignore_index=True)


bigdf.head()

f001
f002
f003
f004
f005
m001
m002
m003
m004
m005


,sec,f0,rms,c,probv,voiced,vuv,file
0,0.02,96.774194,0.000077,0.301661,0.015029,False,uv,f001001_01.wav
1,0.03,129.032258,0.000094,0.360428,0.026731,False,uv,f001001_01.wav
2,0.04,169.014085,0.000095,0.303916,0.015369,False,uv,f001001_01.wav
3,0.05,342.857143,0.000100,0.354015,0.025114,False,uv,f001001_01.wav
4,0.06,244.897959,0.000102,0.342097,0.022356,False,uv,f001001_01.wav


In [ ]:
print(f"N = {len(bigdf)}")
print(bigdf.groupby('vuv').c.mean())
print(bigdf.groupby('vuv').rms.mean())

test_data = bigdf.groupby('vuv').sample(10000,replace=True)
sns.kdeplot(test_data,x="rms",y="c",hue=("vuv"))

In [ ]:
from sklearn.linear_model import LogisticRegression

def warn(*args, **kwargs):   # this is a hack to avoid getting so many printed warning messages
    pass
warnings.warn = warn

# select a test set randomly - no priors, each phone is equally likely
test_data = bigdf.groupby('vuv').sample(500000,replace=True)
test_data['vuv'] = test_data.apply(lambda row: "uv" if row['rms'] < 0.19 and row['vuv'] == 'v' else row['vuv'], axis=1)

X = test_data[['rms','c']].to_numpy()
lm = LogisticRegression(class_weight='balanced').fit(X,test_data["vuv"])
score = lm.score(X,test_data['vuv'])

print(score)
print(f"model is:  {lm.intercept_[0]:3.2f} + {lm.coef_[0,0]:3.2f}*rms + {lm.coef_[0,1]:3.2f}*c")

In [ ]:
test_data = bigdf.groupby('vuv').sample(10)
odds = np.exp(-7.2 + (9.45*test_data.rms) + (10*test_data.c))  # logistic formula
test_data['pv'] = odds / (1 + odds)
test_data['voiced'] = test_data['pv'].apply(lambda x: 'true' if x >0.5 else 'false')

test_data


In [ ]:
df = phon.get_f0("sf3_cln.wav")